In [1]:
# The code was removed by Watson Studio for sharing.

# Capstone Project - The Battle of the Neighborhoods

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

The city of **Santo Domingo** officially Santo Domingo de Guzmán, is the capital and largest city in the [Dominican Republic](https://en.wikipedia.org/wiki/Dominican_Republic) and the largest metropolitan area in the Caribbean by population. As of 2010, the city had a total population of 2,908,607. The city is in the center of economic activity in the Dominican Republic and it is where most of the country's wealth is concentrated. Many national and international firms have their headquarters of regional offices in Santo Domingo. The city attracts many international firms and franchises such as Ikea, Goldcorp and Barrick due to its location and economic stability. 

The city's economic growth can be witnessed in the extensive vertical growth experienced across many of its neighborhoods. Santo Domingo has a thriving middle class contrasting with the significant pockets of poverty that remain as challenges for the future. 

**Brief History about Santo Domingo:** Founded by Bartholomew Columbus in 1496, on the east bank of the Ozama River and then moved by Nicolás de Ovando in 1502 to the west bank of the river, the city is the oldest continuously inhabited European settlement in the Americas, and was the first seat of the Spanish colonial rule in the New World. Santo Domingo is the site of the first university, cathedral, castle, monastery, and fortress in the New World. The city's Colonial Zone was declared as a World Heritage Site by UNESCO.

### Problem Description:  

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Italian restaurant** in Santo Domingo, Dominican Republic.

Since there are lots of restaurants in Santo Domingo we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Italian Restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

### Target Audience:  
The information provided by this report would be useful for stakeholders who are interested in opening an Italian Restaurant in Santo Domingo.

### Success Criteria:   
The aim of this report is to study and analyze the neighborhoods of Santo Domingo city to gather meaningful information. That information can be used to find out neighborhoods with earning potential for an Italian Restaurant and serve a a data point so that stakeholders can take a more informed decision. 

#### Importing Libraries

In [2]:
import numpy as np # handle data into vectorized manner
import pandas as pd # data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # handle json files
from pandas.io.json import json_normalize # transform JSON file to pandas dataframe
import requests # handle requests
import csv

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Clustering Model
from sklearn.cluster import KMeans

!pip install folium==0.5.0 --quiet
import folium # map rendering library

print('Libraries imported!')

Libraries imported!


## Data:<a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:

- number of existing restaurants in the neighborhood (any type of restaurant)
- number of and distance to Italian restaurants in the neighborhood, if any
- distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:

- centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
- number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
- coordinate of Santo Domingo center will be obtained using **Google Maps API geocoding** of well known Santo Domingo location (Parque La Lira, Lope de Vega, Santo Domingo)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candicate neighborhoods. We will create a grid of cells covering our area of interest which is aproximately 3x3 kilometers centered around Santo Domingo City Center. 

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Parque La Lira, Santo Domingo, Dominican Republic'
santodom_center = get_coordinates(api_key, address)
print('Coordinate of {}: {}'.format(address, santodom_center))

Coordinate of Parque La Lira, Santo Domingo, Dominican Republic: [18.4691064, -69.93190109999999]


Now that we have the coordinate of Parque La Lira, Lope de Vega in Santo Domingo let's create a grid of area candidates, equally spaced, centered around city center and within ~6km from the Park. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in Lat/Lon degrees). Then we'll project those coordinates back to latitude/longitude degrees to be whon on Folium map. So let's create functions to convert between [WGS84](https://en.wikipedia.org/wiki/World_Geodetic_System) spherical coordinate system (latitude/longitude degrees) and [UTM Cartesian coordinate system](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system) (X/Y coordinates in meters). PROJ is a generic coordinate transformation sofware that transforms geospacial coordinates from one coordinate reference system (CRS) to another. This includes cartographic projections as well as geodetic transformations. 

In [5]:
!pip install shapely --quiet
import shapely.geometry

!pip install pyproj --quiet
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=19, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=19, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)


print('Coordinate transformation check')
print('-'*30)
print('Santo Domingo Cente longitude={}, latitude={}'.format(santodom_center[1], santodom_center[0]))
x, y = lonlat_to_xy(santodom_center[1], santodom_center[0])
print('Santo Domingo Center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Santo Domingo Center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
------------------------------
Santo Domingo Cente longitude=-69.93190109999999, latitude=18.4691064
Santo Domingo Center UTM X=401607.160447531, Y=2042340.420071079
Santo Domingo Center longitude=-69.93190109999999, latitude=18.4691064


Let's create a heagonal grid of cells: we offset every other row, and adjust vertical row spacing so that every cell center is equally distant from all it's neighbors.

In [6]:
sd_center_x, sd_center_y = lonlat_to_xy(santodom_center[1], santodom_center[0]) # City center in Cartesian coordinates


k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = sd_center_x - 3000
x_step = 600
y_min = sd_center_y - 2000 - (int(21/k)*k*600 - 6000)/2
y_step = 600 * k 


latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(sd_center_x, sd_center_y, x, y)
        if (distance_from_center <= 3001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

92 candidate neighborhood centers generated.


Let's visualize the data so far: city center location and candidate neighborhood centers:

In [7]:
!pip install folium --quiet
import folium

In [8]:
map_stodgo = folium.Map(location=santodom_center, zoom_start=13)
folium.Marker(santodom_center, popup='Parque la Lira').add_to(map_stodgo)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_stodgo) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_stodgo)
    #folium.Marker([lat, lon]).add_to(map_stodgo)
map_stodgo

Now that we have coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~3km from Santo Domingo Center.

Let's now use Google Maps API to get approximate addresses of those locations.

In [9]:
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(api_key, santodom_center[0], santodom_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(santodom_center[0], santodom_center[1], addr))

Reverse geocoding check
-----------------------
Address of [18.4691064, -69.93190109999999] is: Parque La Lira, Av. Lope de Vega, Santo Domingo, Dominican Republic


In [10]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Dominican Republic', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [11]:
addresses

['Calle Dr José Espaillat Rodríguez 24, Santo Domingo',
 'Calle República del Ecuador 6, Santo Domingo',
 'Edificio Gregg, Calle Turey 252, Santo Domingo 10118',
 'Unnamed Road, Santo Domingo',
 'Dominican Republic',
 'Calle F. Pichardo 4, Santo Domingo 10112',
 'Calle Los Laureles 5, Santo Domingo',
 'Av. Anacaona 37, Santo Domingo',
 'Calle 8 35, Santo Domingo',
 'Edificio 1, Calle Fray Cipriano de Utrera, Santo Domingo',
 'Calle Juan Bautista Pérez, Santo Domingo',
 'Dominican Republic',
 'Av. George Washington 555, Santo Domingo',
 'Av 27 de Febrero 436, Santo Domingo 10114',
 'Calle Dionisio Valera 5, Santo Domingo 10114',
 'Calle Bohechio 13, Santo Domingo',
 'Calle Francisco Moreno 18, Santo Domingo 10112',
 'Av. José Contreras 197, Santo Domingo',
 'Av. Dr. Bernardo Correa y Cidrón 65, Santo Domingo',
 'Av. Independencia Proximo C/ Huascar Tejada, Santo Domingo',
 'Av. George Washington 557, Santo Domingo',
 'Av. George Washington 505, Santo Domingo',
 'Panamericana de Producci

Let's now place all this into a Pandas dataframe.

In [12]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"Calle Dr José Espaillat Rodríguez 24, Santo Do...",18.445217,-69.943135,400407.160448,2.039703e+06,2897.479354
1,"Calle República del Ecuador 6, Santo Domingo",18.445245,-69.937454,401007.160448,2.039703e+06,2704.697138
2,"Edificio Gregg, Calle Turey 252, Santo Domingo...",18.445273,-69.931772,401607.160448,2.039703e+06,2637.306696
3,"Unnamed Road, Santo Domingo",18.445301,-69.926091,402207.160448,2.039703e+06,2704.697138
4,Dominican Republic,18.445329,-69.920410,402807.160448,2.039703e+06,2897.479354
5,"Calle F. Pichardo 4, Santo Domingo 10112",18.449870,-69.951683,399507.160448,2.040223e+06,2982.384464
6,"Calle Los Laureles 5, Santo Domingo",18.449899,-69.946002,400107.160448,2.040223e+06,2595.114081
7,"Av. Anacaona 37, Santo Domingo",18.449927,-69.940320,400707.160448,2.040223e+06,2301.003497
8,"Calle 8 35, Santo Domingo",18.449955,-69.934639,401307.160448,2.040223e+06,2138.835452
9,"Edificio 1, Calle Fray Cipriano de Utrera, San...",18.449983,-69.928957,401907.160448,2.040223e+06,2138.835452


Saving Dataframe

In [13]:
project.save_data(file_name="santo_domingo_neigh.csv", data = df_locations.to_csv(index=False), overwrite=True)
print('Dataframe Saved as CSV!')

Dataframe Saved as CSV!


### Foursquare

Now that we have our location cnadidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants -coffee shops, pizza places, bakeries, etc.. are not direct competitors so we don't care about those. So we will include in our list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Food Truck' in category name. 

In [14]:
# The code was removed by Watson Studio for sharing.

In [15]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

italian_restaurant_categories = ['4bf58dd8d48988d110941735','55a5a1ebe4b013909087cbb6','55a5a1ebe4b013909087cb7c',
                                 '55a5a1ebe4b013909087cba7','55a5a1ebe4b013909087cba1','55a5a1ebe4b013909087cba4',
                                 '55a5a1ebe4b013909087cb95','55a5a1ebe4b013909087cb89','55a5a1ebe4b013909087cb9b',
                                 '55a5a1ebe4b013909087cb98','55a5a1ebe4b013909087cbbf','55a5a1ebe4b013909087cb79',
                                 '55a5a1ebe4b013909087cbb0','55a5a1ebe4b013909087cbb3','55a5a1ebe4b013909087cb74',
                                 '55a5a1ebe4b013909087cbaa','55a5a1ebe4b013909087cb83','55a5a1ebe4b013909087cb8c',
                                 '55a5a1ebe4b013909087cb92','55a5a1ebe4b013909087cb8f','55a5a1ebe4b013909087cb86',
                                 '55a5a1ebe4b013909087cbb9','55a5a1ebe4b013909087cb7f','55a5a1ebe4b013909087cbbc',
                                 '55a5a1ebe4b013909087cb9e','55a5a1ebe4b013909087cbc2','55a5a1ebe4b013909087cbad']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [16]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found italian restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    italian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, client_id, client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_italian = is_restaurant(venue_categories, specific_filter=italian_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_italian, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_italian:
                    italian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, italian_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
italian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('italian_restaurants_350.pkl', 'rb') as f:
        italian_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, italian_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('italian_restaurants_350.pkl', 'wb') as f:
        pickle.dump(italian_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Restaurant data loaded.


In [17]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Italian restaurants:', len(italian_restaurants))
print('Percentage of Italian restaurants: {:.2f}%'.format(len(italian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 333
Total number of Italian restaurants: 33
Percentage of Italian restaurants: 9.91%
Average number of restaurants in neighborhood: 3.119565217391304


In [18]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4c09013b6071a593c416dd32', 'Sólo Pastelitos', 18.444882151937406, -69.93649744666435, 'Av. Independencia, 10117 Santo Domingo, República Dominicana', 108, False, 401107.9580562764, 2039662.43457014)
('4bf41a2998ac0f476d9563a8', 'Adrian Tropical', 18.44327234147098, -69.93993857413383, 'Av. Independencia #2047, 10115 Santo Domingo, República Dominicana', 342, False, 400743.631847883, 2039486.1867768797)
('4f8c4fe6e4b0cec3ade5b465', 'De Víctor Heredia Comedor', 18.44365468373122, -69.93835184090811, 'Av. Independencia, República Dominicana', 200, False, 400911.42024362675, 2039527.6251363123)
('4c3b2488d94dc9b6b847ffe7', 'Refrescos Nacionales, CxA', 18.448305993031457, -69.9308967590332, 'Av.Independencia, Santo Domingo, República Dominicana', 350, False, 401701.3680802624, 2040038.2431123008)
('542627b4498e8525dc6a29e0', 'Mila Restaurant', 18.450213553953567, -69.94968167213474, 'República Dominicana', 214, False, 399718.72577764257, 204

In [19]:
print('List of Italian restaurants')
print('---------------------------')
for r in list(italian_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(italian_restaurants))

List of Italian restaurants
---------------------------
('504ab07fe4b0b0e067026789', 'Il Capriccio Pizzeria', 18.453472903511916, -69.94311094869849, 'Ave. Sarasota (Esq. Bohechio), Santo Domingo, República Dominicana', 126, True, 400414.4885877851, 2040616.6550423168)
('53823645498ea6ffdd043046', 'La Lasagna', 18.455292975176114, -69.94067030194486, 'Av. Sarasota 71, 10111 Santo Domingo, República Dominicana', 342, True, 400673.2681891406, 2040816.7108418888)
('4dff79d8d4c00c69c14c8bf8', 'La Locanda', 18.457720370519283, -69.9376874917774, 'Pedro A. Bobea, 10111 Santo Domingo, República Dominicana', 334, True, 400989.64186132746, 2041083.6755606465)
('4c071c6e8b4520a1df898697', 'Bella Italia', 18.456010629452443, -69.92958814616736, 'José Contreras 165 (Av. Jiménez Moya), Santo Domingo, República Dominicana', 279, True, 401843.94003241457, 2040890.0745855086)
('53659468498e6483754160bc', 'il Giardino', 18.456896282364358, -69.93395978904401, 'Jardines del Embajador #10A, 10108 Santo D

In [20]:
print('Restaurants around location')
print('---------------------------')
for i in range(0, 10):
    rs = location_restaurants[i]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 1: 
Restaurants around location 2: Sólo Pastelitos, De Víctor Heredia Comedor
Restaurants around location 3: 
Restaurants around location 4: 
Restaurants around location 5: 
Restaurants around location 6: Mila Restaurant, Applebee's, 100 Montaditos, Shanghai Kitchen Dim Sum Bar, SBG Kitchen, Yao Asian Cuisine, P. F.  Chang's
Restaurants around location 7: BamBam Sushi, Carne & Co.
Restaurants around location 8: Relax
Restaurants around location 9: 
Restaurants around location 10: Metro Fast Food


Let's now see all the collected restaurants in our area of interest on map, and let's also show Italian restaurants in different color.

In [21]:
map_sd = folium.Map(location=santodom_center, zoom_start=13)
folium.Marker(santodom_center, popup='Parque la Lira').add_to(map_sd)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_italian = res[6]
    color = 'red' if is_italian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_sd)
map_sd

Looking good. So now we have all the restaurants in area within few kilometers from Parque la Lira, and we know which ones are Italian restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Italian restaurant!

## Methodology:<a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Berlin that have low restaurant density, particularly those with low number of Italian restaurants. We will limit our analysis to area ~3km around city center.

In first step we have collected the required data: location and type (category) of every restaurant within 3km from Santo Domingo City center. We have also identified Italian restaurants (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of 'restaurant density' across different areas of Santo Domingo - we will use heatmaps to identify a few promising areas close to center with low number of restaurants in general (and no Italian restaurants in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create clusters of locations that meet some basic requirements established in discussion with stakeholders: we will take into consideration locations with no more than two restaurants in radius of 250 meters, and we want locations without Italian restaurants in radius of 400 meters. We will present map of all such locations but also create clusters (using k-means clustering) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis:<a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the number of restaurants in every area candidate:

In [22]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=300m: 3.119565217391304


,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area
0,"Calle Dr José Espaillat Rodríguez 24, Santo Do...",18.445217,-69.943135,400407.160448,2.039703e+06,2897.479354,0
1,"Calle República del Ecuador 6, Santo Domingo",18.445245,-69.937454,401007.160448,2.039703e+06,2704.697138,2
2,"Edificio Gregg, Calle Turey 252, Santo Domingo...",18.445273,-69.931772,401607.160448,2.039703e+06,2637.306696,0
3,"Unnamed Road, Santo Domingo",18.445301,-69.926091,402207.160448,2.039703e+06,2704.697138,0
4,Dominican Republic,18.445329,-69.920410,402807.160448,2.039703e+06,2897.479354,0
5,"Calle F. Pichardo 4, Santo Domingo 10112",18.449870,-69.951683,399507.160448,2.040223e+06,2982.384464,7
6,"Calle Los Laureles 5, Santo Domingo",18.449899,-69.946002,400107.160448,2.040223e+06,2595.114081,2
7,"Av. Anacaona 37, Santo Domingo",18.449927,-69.940320,400707.160448,2.040223e+06,2301.003497,1
8,"Calle 8 35, Santo Domingo",18.449955,-69.934639,401307.160448,2.040223e+06,2138.835452,0
9,"Edificio 1, Calle Fray Cipriano de Utrera, San...",18.449983,-69.928957,401907.160448,2.040223e+06,2138.835452,1


OK, now let's calculate the distance to nearest Italian restaurant from every area candidate center (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [23]:
distances_to_italian_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in italian_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_italian_restaurant.append(min_distance)

df_locations['Distance to Italian restaurant'] = distances_to_italian_restaurant

In [24]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Italian restaurant
0,"Calle Dr José Espaillat Rodríguez 24, Santo Do...",18.445217,-69.943135,400407.160448,2.039703e+06,2897.479354,0,913.571059
1,"Calle República del Ecuador 6, Santo Domingo",18.445245,-69.937454,401007.160448,2.039703e+06,2704.697138,2,1088.952851
2,"Edificio Gregg, Calle Turey 252, Santo Domingo...",18.445273,-69.931772,401607.160448,2.039703e+06,2637.306696,0,1210.347672
3,"Unnamed Road, Santo Domingo",18.445301,-69.926091,402207.160448,2.039703e+06,2704.697138,0,1241.292063
4,Dominican Republic,18.445329,-69.920410,402807.160448,2.039703e+06,2897.479354,0,1489.523511
5,"Calle F. Pichardo 4, Santo Domingo 10112",18.449870,-69.951683,399507.160448,2.040223e+06,2982.384464,7,989.152355
6,"Calle Los Laureles 5, Santo Domingo",18.449899,-69.946002,400107.160448,2.040223e+06,2595.114081,2,499.628476
7,"Av. Anacaona 37, Santo Domingo",18.449927,-69.940320,400707.160448,2.040223e+06,2301.003497,1,490.749270
8,"Calle 8 35, Santo Domingo",18.449955,-69.934639,401307.160448,2.040223e+06,2138.835452,0,771.441074
9,"Edificio 1, Calle Fray Cipriano de Utrera, San...",18.449983,-69.928957,401907.160448,2.040223e+06,2138.835452,1,670.333844


In [25]:
print('Average distance to closest Italian restaurant from each area center:', df_locations['Distance to Italian restaurant'].mean())

Average distance to closest Italian restaurant from each area center: 650.979663976205


OK, so **on average Italian restaurant can be found within ~650m** from every area center candidate. That's fairly close, so we need to filter our areas carefully!

Let's crete a map showing heatmap / density of restaurants and try to extract some meaningfull info from that.

In [26]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

italian_latlons = [[res[2], res[3]] for res in italian_restaurants.values()]

In [27]:
from folium import plugins
from folium.plugins import HeatMap

map_sd = folium.Map(location=santodom_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_sd) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_sd)
folium.Marker(santodom_center).add_to(map_sd)
folium.Circle(santodom_center, radius=1000, fill=False, color='white').add_to(map_sd)
folium.Circle(santodom_center, radius=2000, fill=False, color='white').add_to(map_sd)
folium.Circle(santodom_center, radius=3000, fill=False, color='white').add_to(map_sd)
map_sd

Looks like a few pockets of low restaurant density closest to city center can be found north, north-east and east from Parque la Lira.

Let's create another heatmap map showing heatmap/density of Italian restaurants only.

In [28]:
map_sd = folium.Map(location=santodom_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_sd) #cartodbpositron cartodbdark_matter
HeatMap(italian_latlons).add_to(map_sd)
folium.Marker(santodom_center).add_to(map_sd)
folium.Circle(santodom_center, radius=1000, fill=False, color='white').add_to(map_sd)
folium.Circle(santodom_center, radius=2000, fill=False, color='white').add_to(map_sd)
folium.Circle(santodom_center, radius=3000, fill=False, color='white').add_to(map_sd)
map_sd

This map is not so 'hot' (Italian restaurants represent a subset of ~10% of all restaurants in Santo Domingo) but it also indicates higher density of existing Italian restaurants directly north and south from Parque la Lira, with closest pockets of low Italian restaurant density positioned north-east, south-east and south from city center.

OK. Let us now filter those locations: we're interested only in locations with no more than two restaurants in radius of 250 meters, and no Italian restaurants in radius of 400 meters.

In [29]:
good_res_count = np.array((df_locations['Restaurants in area']<=2))
print('Locations with no more than two restaurants nearby:', good_res_count.sum())

good_ita_distance = np.array(df_locations['Distance to Italian restaurant']>=400)
print('Locations with no Italian restaurants within 400m:', good_ita_distance.sum())

good_locations = np.logical_and(good_res_count, good_ita_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_locations[good_locations]


Locations with no more than two restaurants nearby: 56
Locations with no Italian restaurants within 400m: 61
Locations with both conditions met: 50


Let's see how that looks on a map.

In [30]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_sd = folium.Map(location=santodom_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_sd)
HeatMap(restaurant_latlons).add_to(map_sd)
folium.Circle(santodom_center, radius=2500, color='white', fill=True, fill_opacity=0.6).add_to(map_sd)
folium.Marker(santodom_center).add_to(map_sd)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_sd) 
map_sd

Looking good. We now have a bunch of locations fairly close to Parque la Lira, and we know that each of those locations has no more than two restaurants in radius of 250m, and no Italian restaurant closer than 400m. Any of those locations is a potential candidate for a new Italian restaurant, at least based on nearby competition.

Let's now show those good locations in a form of heatmap:

In [31]:
map_sd = folium.Map(location=santodom_center, zoom_start=14)
HeatMap(good_locations, radius=25).add_to(map_sd)
folium.Marker(santodom_center).add_to(map_sd)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_sd)
map_sd

Looking good. What we have now is a clear indication of zones with low number of restaurants in vicinity, and no Italian restaurants at all nearby.

Let us now cluster those locations to create centers of zones containing good locations. Those zones, their centers and addresses will be the final result of our analysis.

In [32]:
from sklearn.cluster import KMeans

number_of_clusters = 15

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_sd = folium.Map(location=santodom_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_sd)
HeatMap(restaurant_latlons).add_to(map_sd)
folium.Circle(santodom_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_sd)
folium.Marker(santodom_center).add_to(map_sd)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_sd) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_sd)
map_sd

Not bad - our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

Addresses of those cluster centers will be a good starting point for exploring the neighborhoods to find the best possible location based on neighborhood specifics.

Let's see those zones on a city map without heatmap, using shaded areas to indicate our clusters:

In [33]:
map_sd = folium.Map(location=santodom_center, zoom_start=14)
folium.Marker(santodom_center).add_to(map_sd)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_sd)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_sd)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_sd) 
map_sd

Finally, let's reverse geocode those candidate area centers to get the addresses which can be presented to stakeholders.

In [34]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr = get_address(api_key, lat, lon).replace(', Dominican Republic', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, sd_center_x, sd_center_y)
    print('{}{} => {:.1f}km from Parque la Lira'.format(addr, ' '*(50-len(addr)), d/1000))
    

Addresses of centers of areas recommended for further analysis

Helipuerto El Caribe-CDN, Santo Domingo            => 2.8km from Parque la Lira
Autopista 30 de Mayo, Santo Domingo                => 2.5km from Parque la Lira
Av. Kennedy Con Av. Maximo Gomez, Santo Domingo    => 2.3km from Parque la Lira
Calle Rafael Augusto Sánchez 124, Santo Domingo    => 1.5km from Parque la Lira
km 7, Residencial Independencia I, Santo Domingo   => 2.4km from Parque la Lira
Calle Wifredo García 13, Santo Domingo             => 2.6km from Parque la Lira
Calle César Nicolás Penson 76, Santo Domingo 10205 => 2.6km from Parque la Lira
Av. José Contreras 41, Santo Domingo               => 1.6km from Parque la Lira
Calle 14 10, Santo Domingo                         => 2.1km from Parque la Lira
Calle Guayacanes 80, Santo Domingo                 => 2.4km from Parque la Lira
Av. George Washington 555, Santo Domingo           => 2.9km from Parque la Lira
Av. Anacaona 59, Santo Domingo                     => 2.

This concludes our analysis. We have created 15 addresses representing centers of zones containing locations with low number of restaurants and no Italian restaurants nearby, all zones being fairly close to city center. Although zones are shown on map with a radius of ~500 meters (green circles), their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential restaurant locations. Most of the zones are located in Evaristo Morales and Los Praditos boroughs, which we have identified as interesting due to being high denisty with middle class, fairly close to city center and well connected by public transport.

In [35]:
map_sd = folium.Map(location=santodom_center, zoom_start=14)
folium.Circle(santodom_center, radius=50, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_sd)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]], popup=addr).add_to(map_sd) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.05).add_to(map_sd)
map_sd

## Results and Discussion:<a name="results"></a>

Our analysis shows that although there is a considerable number of restaurants in Santo Domingo (333 in our initial area of interest), there are pockets of low restaurant density fairly close to city center. Highest concentration of restaurants was detected north and south from Parque la Lira with higher density of existing Italian restaurants directly north and south from Parque la Lira, with closest pockets of low Italian restaurant density positioned north-east, south-east and south from city center.

Since Santo Domingo is a fairly small city we filtered so that those areas with more than two restaurants in radius of 250m and those with an Italian restaurant closer than 400m were removed.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 15 zones containing largest number of potential new restaurant locations based on number of and distance to existing venues - both restaurants in general and Italian restaurants particularly. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to Santo Domingo center but not crowded with existing restaurants (particularly Italian) - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion:<a name="conclusion"></a>

Purpose of this project was to identify areas in Santo Domingo close to center with low number of restaurants (particularly Italian restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Italian restaurant. By calculating restaurant density distribution from Foursquare data we have first identified general boroughs and generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.